In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 22420



In [2]:
sas.submitLST(f"""
    data cars;
        set sashelp.cars;
        if strip(model) in ('TSX 4dr', 'TL 4dr', 'MDX') then do;
            msrp = .;
            invoice = .;
            mpg_city = .;
            mpg_highway = .;
        end;
        if strip(model) in ('3.5 RL 4dr') then do;
            type = '';
            origin = '';
        end;
        keep model type origin msrp invoice mpg_city mpg_highway;
    run;
              
    proc sort data=cars nodupkey;
        by model;
    run;

    proc print data=cars (obs=5);
    run;
""")

Obs,Model,Type,Origin,MSRP,Invoice,MPG_City,MPG_Highway
1,3.5 RL 4dr,,,"$43,755","$39,014",18,24
2,3.5 RL w/Navigation 4dr,Sedan,Asia,"$46,100","$41,100",18,24
3,300M 4dr,Sedan,USA,"$29,865","$27,797",18,27
4,300M Special Edition 4dr,Sedan,USA,"$33,295","$30,884",18,27
5,325Ci 2dr,Sedan,Europe,"$30,795","$28,245",20,29


In [8]:
# proc means, with out, noprint, maxdec
# by gives multiple tables, data need to be sorted on the by var, lol!
# var controls observations in the table
sas.submitLST(f"""
    PROC SORT DATA = cars OUT = _sorted_for_means;
    BY type;

    title;
    PROC MEANS DATA = _sorted_for_means MAXDEC = 2 noprint; 
        where mpg_city > 1;
        BY type; 
        VAR msrp invoice;
        output out = _means_out 
            mean(msrp invoice) = mean_msrp mean_invoice;
    RUN;
              
    PROC PRINT DATA = _means_out;
    RUN;
""")

Obs,Type,_TYPE_,_FREQ_,mean_msrp,mean_invoice
1,,0,1,"$43,755","$39,014"
2,Hybrid,0,3,"$19,920","$18,429"
3,SUV,0,59,"$34,754","$31,596"
4,Sedan,0,256,"$29,656","$27,285"
5,Sports,0,49,"$53,387","$48,473"
6,Truck,0,24,"$24,941","$22,617"
7,Wagon,0,30,"$28,841","$26,646"


In [6]:
# class has one obs for each interaction of the by variables
sas.submitLST(f"""
    * Calculate means by CustomerID, output sum and mean to new data set;
    PROC MEANS NOPRINT DATA = sales;
        BY CustID;
        class saledate;
        VAR Petunia SnapDragon Marigold;
        OUTPUT OUT = totals 
            N(Petunia SnapDragon Marigold) = NP NSD NM 
        ;
    
    PROC PRINT DATA = totals;
        TITLE 'Sum of Flower Data over Customer ID';
    RUN;
""")


28                                                         The SAS System                          Tuesday, May  7, 2024 01:32:00 PM

201        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
201      !  graphics on / outputfmt=png;
202        
203        
204            * Calculate means by CustomerID, output sum and mean to new data set;
205            PROC MEANS NOPRINT DATA = sales;
206                BY CustID;
ERROR: Variable CUSTID not found.
207                class saledate;
ERROR: Variable SALEDATE not found.
208                VAR Petunia SnapDragon Marigold;
ERROR: Variable PETUNIA not found.
ERROR: Variable SNAPDRAGON not found.
ERROR: Variable MARIGOLD not found.
209                OUTPUT OUT = totals
210                    N(Petunia SnapDragon Marigold) = NP NSD NM
ERROR: Variable PETUNIA not found.
ERROR: Variable SNAPDRAGON not found.
ERROR: Variable MARIGOLD not found.
211                ;
212

In [13]:
# proc summary


### nway